### Imports

In [ ]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "VLCS"
MODEL_NAME = "WinKawaks/vit-tiny-patch16-224"

Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [ ]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [ ]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

Initialized experiment directories for VLCS
Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\VLCS\logs
Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\VLCS\checkpoints
Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\VLCS\plots


### Setup

In [ ]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [ ]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== LODO: Leaving out domain 'VOC2007' ===


Evaluating: 100%|██████████| 27/27 [00:26<00:00,  1.01it/s]


[VOC2007] Epoch 1/5 | Train - Loss: 0.6864, Cls: 0.6735, GRQO: 0.0129, Acc: 0.7371 | Val - Loss: 0.6388, Cls: 0.6362, GRQO: 0.0025, Acc: 0.7595
[VOC2007] New best val acc: 0.7595


Evaluating: 100%|██████████| 27/27 [00:24<00:00,  1.10it/s]


...

[SUN09] Epoch 3/5 | Train - Loss: 0.2541, Cls: 0.2510, GRQO: 0.0031, Acc: 0.9112 | Val - Loss: 0.7010, Cls: 0.6998, GRQO: 0.0012, Acc: 0.7556
[SUN09] New best val acc: 0.7556


Evaluating: 100%|██████████| 26/26 [00:24<00:00,  1.06it/s]


[SUN09] Epoch 4/5 | Train - Loss: 0.1562, Cls: 0.1535, GRQO: 0.0026, Acc: 0.9514 | Val - Loss: 0.7539, Cls: 0.7530, GRQO: 0.0009, Acc: 0.7751
[SUN09] New best val acc: 0.7751


Evaluating: 100%|██████████| 26/26 [00:24<00:00,  1.06it/s]

[SUN09] Epoch 5/5 | Train - Loss: 0.0870, Cls: 0.0846, GRQO: 0.0023, Acc: 0.9752 | Val - Loss: 0.9743, Cls: 0.9736, GRQO: 0.0007, Acc: 0.7413
[SUN09] Best Acc: 0.7751
------------------------------------------------------------
LODO finished | Mean Acc: 0.8049
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\VLCS\logs\lodo_summary_20251010_090225.json


### Baseline

In [ ]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([5, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'VOC2007' ===
[VOC2007] Epoch 1/5 | Train - Loss: 0.5627, Acc: 0.7850 | Val Acc: 0.8027
[VOC2007] Epoch 2/5 | Train - Loss: 0.2946, Acc: 0.8980 | Val Acc: 0.8001
[VOC2007] Epoch 3/5 | Train - Loss: 0.1577, Acc: 0.9490 | Val Acc: 0.7941
[VOC2007] Epoch 4/5 | Train - Loss: 0.0579, Acc: 0.9830 | Val Acc: 0.7983
[VOC2007] Epoch 5/5 | Train - Loss: 0.0246, Acc: 0.9937 | Val Acc: 0.7645
[VOC2007] Best Val Acc: 0.8027
------------------------------------------------------------
Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([5, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'LabelMe' ===
[LabelMe] Epoch 1/5 | Train - Loss: 0.4921, Acc: 0.8214 | Val Acc: 0.6525
[LabelMe] Epoch 2/5 | Train - Loss: 0.1861, Acc: 0.9373 | Val Acc: 0.6404
[LabelMe] Epoch 3/5 | Train - Loss: 0.0984, Acc: 0.9700 | Val Acc: 0.6803
[LabelMe] Epoch 4/5 | Train - Loss: 0.0277, Acc: 0.9936 | Val Acc: 0.6691
[LabelMe] Epoch 5/5 | Train - Loss: 0.0109, Acc: 0.9984 | Val Acc: 0.6487
[LabelMe] Best Val Acc: 0.6803
------------------------------------------------------------
Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([5, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'Caltech101' ===
[Caltech101] Epoch 1/5 | Train - Loss: 0.6220, Acc: 0.7645 | Val Acc: 0.9859
[Caltech101] Epoch 2/5 | Train - Loss: 0.3137, Acc: 0.8866 | Val Acc: 0.9767
[Caltech101] Epoch 3/5 | Train - Loss: 0.1605, Acc: 0.9465 | Val Acc: 0.9866
[Caltech101] Epoch 4/5 | Train - Loss: 0.0611, Acc: 0.9821 | Val Acc: 0.9802
[Caltech101] Epoch 5/5 | Train - Loss: 0.0306, Acc: 0.9918 | Val Acc: 0.9802
[Caltech101] Best Val Acc: 0.9866
------------------------------------------------------------
Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([5, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'SUN09' ===
[SUN09] Epoch 1/5 | Train - Loss: 0.5781, Acc: 0.7830 | Val Acc: 0.7233
[SUN09] Epoch 2/5 | Train - Loss: 0.2651, Acc: 0.9060 | Val Acc: 0.7562
[SUN09] Epoch 3/5 | Train - Loss: 0.1316, Acc: 0.9570 | Val Acc: 0.7239
[SUN09] Epoch 4/5 | Train - Loss: 0.0521, Acc: 0.9852 | Val Acc: 0.7465
[SUN09] Epoch 5/5 | Train - Loss: 0.0144, Acc: 0.9976 | Val Acc: 0.7690
[SUN09] Best Val Acc: 0.7690
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-tiny-patch16-224) finished | Mean Acc: 0.8097
